In [1]:
# Base
import os
import pandas as pd
import numpy as np

from ml_logic.main import preprocess, preprocess_test

# AutoML with TPOT
import tpot

/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
target_chosen = "nb_departures"
X_processed_df, y_processed_df, preprocessor = preprocess(target_chosen)
X_test_processed, y_test_processed=preprocess_test(preprocessor, target_chosen)

Raw data imported


/home/anson/code/G-Dolle/DIVVY_BIKE/ml_logic/cleaning.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_reduced.rename(columns={"name":"station_name"}, inplace=True)


Data cleaned and merged
features and target dataframes created
features preprocessed
nb_departures picked as target
Preprocessing of Training set is done
Test Raw data imported


/home/anson/code/G-Dolle/DIVVY_BIKE/ml_logic/cleaning.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stations_reduced.rename(columns={"name":"station_name"}, inplace=True)


Test Data cleaned and merged
Test features and target dataframes created
nb_departures picked as target
Preprocessing of test set is done


In [3]:
X_processed_df.shape, X_test_processed.shape

((52546, 71), (44841, 71))

In [8]:
# generations=20, population_size=20
model = tpot.TPOTRegressor(generations=20, population_size=20, scoring='neg_mean_squared_error',
        verbosity=2, n_jobs=-1)
model.fit(X_processed_df, y_processed_df)
model.score(X_test_processed, y_test_processed)

                                                                                
Generation 1 - Current best internal CV score: -2.9531885209749875
                                                                               
Generation 2 - Current best internal CV score: -2.831324647479033
                                                                               
Generation 3 - Current best internal CV score: -2.831324647479033
                                                                               
Generation 4 - Current best internal CV score: -2.8209636530407343
                                                                                
Generation 5 - Current best internal CV score: -2.8209636530407343
                                                                                
Generation 6 - Current best internal CV score: -2.8209636530407343
                                                                                
Generation 7 - Current best internal

/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


-1.8302977502340134

In [14]:
# generations=20, population_size=20
model = tpot.TPOTRegressor(generations=20, population_size=20, scoring='neg_mean_squared_error',
        verbosity=2, n_jobs=-1)
model.fit(X_processed_df, y_processed_df)
model.score(X_test_processed, y_test_processed)

                                                                                
Generation 1 - Current best internal CV score: -2.953188520974987
                                                                               
Generation 2 - Current best internal CV score: -2.9531885209749817
                                                                               
Generation 3 - Current best internal CV score: -2.9531885209749817
                                                                               
Generation 4 - Current best internal CV score: -2.5469190355815785
                                                                                
Generation 5 - Current best internal CV score: -2.5469190355815785
                                                                              
Generation 6 - Current best internal CV score: -2.5469190355815785
                                                                              
Generation 7 - Current best internal CV

/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


-1.7767766193748655

In [19]:
# generations=100, population_size=20
model = tpot.TPOTRegressor(generations=1, population_size=1, scoring='neg_mean_squared_error',
        verbosity=2, n_jobs=-1)
model.fit(X_processed_df, y_processed_df)
model.score(X_test_processed, y_test_processed)

                                                                           
Generation 1 - Current best internal CV score: -4.833979952264002
                                                         
Best pipeline: SGDRegressor(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False), alpha=0.01, eta0=1.0, fit_intercept=True, l1_ratio=1.0, learning_rate=constant, loss=epsilon_insensitive, penalty=elasticnet, power_t=10.0)


/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/metrics/_scorer.py:765: FutureWarning: sklearn.metrics.SCORERS is deprecated and will be removed in v1.3. Please use sklearn.metrics.get_scorer_names to get a list of available scorers and sklearn.metrics.get_metric to get scorer.
  warnings.warn(


-3.164303489334495

In [20]:
# Export the result
model.export('tpot_ml_pipeline.py')

In [33]:
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import LinearSVR
from sklearn.ensemble import StackingRegressor

model_dep = StackingRegressor(
    estimators = [("svr", LinearSVR(C=25.0, dual=True, epsilon=1.0, loss='squared_epsilon_insensitive', tol=0.0001)),
                  ("xbg", XGBRegressor(learning_rate=0.01, max_depth=3, min_child_weight=19, n_estimators=100, n_jobs=1, objective='reg:squarederror', subsample=0.8, verbosity=0))],
    final_estimator = DecisionTreeRegressor(max_depth=3, min_samples_leaf=12, min_samples_split=18))

cv_results_dep=cross_validate(model_dep,X_processed_df, y_processed_df,cv=10,scoring=('neg_mean_squared_error'))
cv_results_dep['test_score'].mean()

/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increas

-3.013921767887427

In [23]:
model_dep = StackingRegressor(
    estimators = [("svr", LinearSVR(C=25.0, dual=True, epsilon=1.0, loss='squared_epsilon_insensitive', tol=0.0001)),
                  ("xbg", XGBRegressor(learning_rate=0.01, max_depth=3, min_child_weight=19, n_estimators=100, n_jobs=1, objective='reg:squarederror', subsample=0.8, verbosity=0))],
    final_estimator = DecisionTreeRegressor(max_depth=3, min_samples_leaf=12, min_samples_split=18))
model_dep.fit(X_processed_df, y_processed_df)

/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/anson/.pyenv/versions/3.10.6/envs/DIVVY_BIKE/lib/python3.10/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increas

StackingRegressor(estimators=[('svr',
                               LinearSVR(C=25.0, epsilon=1.0,
                                         loss='squared_epsilon_insensitive')),
                              ('xbg',
                               XGBRegressor(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            gpu_id=None, grow_...
                                            learning_rate=0.01, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None,
                                            min_child_weight=19, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=1,
                                            num_parallel_tree=None,
                                            predictor=None, random_state=None, ...))],
                  final_estimator=DecisionTreeRegressor(max_depth=3,
                                                        min_samples_leaf=12,
                                                        min_samples_split=18))

In [25]:
model_dep.predict(X_test_processed)

array([0.91521011, 0.91521011, 0.91521011, ..., 1.8983437 , 0.91521011,
       0.91521011])